# Phase 4 Project

**Author: Magali Solimano**

## Overview

Question: What are the top 5 best zip codes for us to invest in?

## Business Understanding

Criteria: Zipcodes with below-median sales price and annual ROI 
(over previous 5 years) in 90th percentile.  
Scope: The analysis focuses on the state of FL, which exploratory data analysis shows 
has a high percentage  
of zipcodes that meet this criteria.

## Data

__Variable Descriptions__

- __RegionID__: Unique ID for each region

- __RegionName__: Zipcode

- __City__: City name

- __State__: State name

- __Metro__: Name of metro city around region

- __CountyName__: County name

- __SizeRank__: Rank by size of region

- __SalePrice__: Median sales price

## Loading data and libraries

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
import itertools

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from pmdarima.arima import auto_arima


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load dataset
df = pd.read_csv('data/zillow_data.csv')
df.head()

In [ ]:
# Examine shape of dataset
df.shape

In [ ]:
# Examine number of unique values for all columns except time series columns
df.iloc[:, 0:7].nunique()

Dataset has over 14,700 entries spanning a twenty-two year period from April 1996 to April 2018. The dataset  
includes 50 states and Washington, DC, and it covers 7,554 cities and 701 metro areas. 

Each row represents a zipcode, which is captured by RegionName in this dataset. Number of rows is  
equal to unique entries for RegionName.

## Exploratory Data Analysis and Visualization

In [ ]:
# Rename columns: RegionName refers to zipcode, and CountyName can be renamed to County
df.rename({'RegionName': 'Zipcode'}, axis='columns', inplace=True)
df.rename({'CountyName': 'County'}, axis='columns', inplace=True)
# Set dtype to string
df.Zipcode = df.Zipcode.astype(str)
df.head()

In [ ]:
# Check for missing values by column
df.isnull().sum()

Metro has missing values. The time series data is also likely to have missing values.

In [ ]:
# Fill Metro na's with 'None'
df.Metro.fillna('None', inplace=True)

In [ ]:
# Check for total missing values
df.isna().sum().sum()

In [ ]:
# Group zipcode by city, county, and state
df[['Zipcode','City','County','State']].sort_values(by=['Zipcode'])

A number of MA's zipcodes in the dataset only have four digits. Looking up the state's  
zipcodes, many start with '0', which appear to have been dropped in this dataset.

In [ ]:
# Insert '0' as first digit in zipcodes with only four digits.
for i in range(len(df)):
    df.Zipcode[i] = df.Zipcode[i].rjust(5, '0')

# Return zipcode min to confirm that value now starts with '0'    
df.Zipcode.min()    

In [ ]:
# Plot value counts by state, county, metro, and city
# Create figure and set size
fig, axes = plt.subplots(nrows=1 , ncols=4, figsize=(15,8))

# Plot value counts by state, metro, and city
df.State.value_counts().head(50).plot(kind='barh', title='State', ax = axes[0])
df.County.value_counts().head(50).plot(kind='barh', title='County', ax = axes[1])
df.Metro.value_counts().head(50).plot(kind='barh', title='Metro Area', ax = axes[2])
df.City.value_counts().head(50).plot(kind='barh', title='City', ax = axes[3])

# Invert y axes
axes[0].invert_yaxis()
axes[1].invert_yaxis()
axes[2].invert_yaxis()
axes[3].invert_yaxis()

# Set title
plt.suptitle('Value Counts by State, County, Metro Area, and City', 
             y=1.01, fontsize=12)

# Format layout
plt.tight_layout()

In [ ]:
# Search for missing data in time series columns
for col in reversed(df.columns):
    if df[col].isna().sum() > 0:
        print(col)
        break

Data appears to be missing for some zipcodes in period leading up to June 2014.  
How many zipcodes are affected?

In [ ]:
# Return zipcodes that have missing data in June 2014
print(len(df[df['2014-06'].isna()]))
df[df['2014-06'].isna()]

- 56 zipcodes have missing data prior to June 2014

In [ ]:
# Return mean and median prices in 2018 and 2013, and calculate percent change
# Mean and median price across all zipcodes in April 2018 and July 2014
print('Mean price, April 2018: $', np.round(df.groupby('Zipcode')['2018-04'].mean().mean()))
print('Median price, April 2018: $', df.groupby('Zipcode')['2018-04'].median().median())
print('-------------------------------------')
# Mean and median price across all zipcodes in April 2013
print('Mean price, April 2013: $', np.round(df.groupby('Zipcode')['2013-04'].mean().mean()))
print('Median price, April 2013: $', df.groupby('Zipcode')['2013-04'].median().median())
print('--------------------------------------')
# Price percentage change, 2018 vs. 2013
print('Mean price, percent change: ', np.round(100*(288040-198100)/198100),'%')
print('Median price, percent change: ', np.round(100*(211731-151200)/151200),'%')

- Mean and median sales prices have increased over 5y period.

In [ ]:
# Calculate ROI metrics
# Cumulative 5-year ROI from 2013 to 2018
df['roi_cum'] = 100*(df['2018-04'] - df['2013-04']) / df['2013-04']

# Annual ROI for most recent 5 full years of data
df['roi_13-14'] = 100*(df['2014-04'] - df['2013-04']) / df['2013-04']
df['roi_14-15'] = 100*(df['2015-04'] - df['2014-04']) / df['2014-04']
df['roi_15-16'] = 100*(df['2016-04'] - df['2015-04']) / df['2015-04']
df['roi_16-17'] = 100*(df['2017-04'] - df['2016-04']) / df['2016-04']
df['roi_17-18'] = 100*(df['2018-04'] - df['2017-04']) / df['2017-04']

# Average annual ROI over 5-year period
df['roi_annual_avg_5y'] = (df['roi_13-14'] + df['roi_14-15'] + 
                           df['roi_15-16'] + df['roi_16-17'] + 
                           df['roi_17-18']) / 5

# Average annual ROI over 3-year period
df['roi_annual_avg_3y'] = (df['roi_15-16'] + 
                        df['roi_16-17'] + df['roi_17-18']) / 3

In [ ]:
# Assess ROI metrics by zipcode
# Return median cumulative ROI for all zipcodes and by each zipcode
print('Median ROI from 2013-2018 (%): ', np.round(df.roi_cum.median(),2))

# Sort by highest average annual ROI
df.sort_values('roi_annual_avg_5y',ascending=False).head(10)[
                                    ['Zipcode','City','State',
                                     '2018-04','roi_cum',
                                     'roi_annual_avg_5y']]

- Zipcodes in CO, CA (2), MI, FL (3), TN, PA, and NY have the highest median  
annual ROI over the 5y period.

In [ ]:
# Descriptive statistics for average annual ROI
print(df.roi_annual_avg_5y.describe())
print('90%: ', df['roi_annual_avg_5y'].quantile(0.90))
print('95%: ', df['roi_annual_avg_5y'].quantile(0.95))
print('99%: ', df['roi_annual_avg_5y'].quantile(0.99))
print('99.9%: ', df['roi_annual_avg_5y'].quantile(0.999))

In [ ]:
# Descriptive statistics for sales price in April 2018
print(df['2018-04'].describe())
print('20%: ', df['2018-04'].quantile(0.20))
print('90%: ', df['2018-04'].quantile(0.90))
print('95%: ', df['2018-04'].quantile(0.95))

In [ ]:
# Plot sales price and average annual ROI for all states
# Create figure and set size
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(8,6))

# Plot average 2018 sales price by state
df.groupby('State')['2018-04'].median().sort_values(
                                                ascending=False).plot(
                                                kind='bar',
                                                color='lightblue',
                                                ax=ax[0])
# Plot average sales price for all states
ax[0].axhline(y=np.nanmedian(df['2018-04']), color='black', 
              linestyle='--')

# Set y-axis label and format y-axis ticks
ax[0].set_ylabel('Sales Price ($)')
ax[0].set(ylim=(0, 800000))
ax[0].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

# Add and format legend
price_labels = ['US Median Sales Price', 'Sales Price']

# Slice list to remove first handle
ax[0].legend(loc='topcenter', frameon=False, labels=price_labels)

# Plot median annual ROI by state
df.groupby('State')['roi_annual_avg_5y'].median().sort_values(
                                                ascending=False).plot(
                                                kind='bar',
                                                color='paleturquoise',
                                                ax=ax[1])

# Plot median annual ROI for all states
ax[1].axhline(y=np.nanmedian(df.roi_annual_avg_5y), color='black', 
             linestyle='--')

# Set y-axis label
ax[1].set_ylabel('ROI (%)')

# Add and format legend
roi_labels = ['US Median Annual ROI', 'Annual ROI']

# Slice list to remove first handle
ax[1].legend(labels = roi_labels, 
             loc='upper right', frameon=False)

# Set title
plt.suptitle('Median Sales Price and Annual ROI by State', fontsize='13',
            y=1.02)

# Format plot
sns.despine()
plt.tight_layout()

In [ ]:
# Boxplots of 2018 median sales price and average annual ROI over 5-yr period

# Create figure and set size
fig, ax = plt.subplots(nrows=1, ncols= 2, figsize = (7,5))

# Plot boxplots
sns.boxplot(data = df['2018-04'], color = 'steelblue', ax=ax[0]).set(
    xlabel='Median Sales Price, 2018')
sns.boxplot(data = df.roi_annual_avg_5y, color='green', ax=ax[1]).set(
    xlabel='ROI, 5-year Annual Average')

# Set y-axis range
ax[0].set(ylim=(0, 1000000))
ax[0].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
ax[1].set(ylim=(-10,25))

# Remove xticklabels and xticks
ax[0].set_xticklabels([])
ax[0].tick_params(bottom=False) 
ax[1].set_xticklabels([])
ax[1].tick_params(bottom=False) 

# Set title
plt.suptitle('Distribution of Sales Price and ROI', y = 1.02)
plt.tight_layout()

In [ ]:
# Sort df by sales price and ROI criteria for business case
# Criteria: sales price between 20th and 50th percentiles and 
# ROI > 90th percentile
df[(df['2018-04'] >= np.percentile(df['2018-04'], 20)) & 
   (df['2018-04'] < np.percentile(df['2018-04'], 50)) & 
   (df['roi_annual_avg_5y'] > 10.73)].sort_values(
    by='roi_annual_avg_5y', ascending=False).head(10)

- Many zipcodes in Florida emerge among top results according to criteria to focus on below-median sales  
price and annual average ROI (over previous 5 years) in the top 90th percentile.

In [ ]:
# Create df for Florida zipcodes
df_fl = df.loc[df['State']=='FL']
df_fl.head()

In [ ]:
# Descriptive statistics for FL zipcodes annual ROI, 5y period
print(df_fl['roi_annual_avg_5y'].describe())
print('90%: ', df_fl['roi_annual_avg_5y'].quantile(0.90))
print('95%: ', df_fl['roi_annual_avg_5y'].quantile(0.95))
print('99%: ', df_fl['roi_annual_avg_5y'].quantile(0.99))
print('99.9%: ', df_fl['roi_annual_avg_5y'].quantile(0.999))

In [ ]:
# Descriptive statistics for FL zipcodes annual ROI, 3y period
print(df_fl['roi_annual_avg_3y'].describe())
print('90%: ', df_fl['roi_annual_avg_3y'].quantile(0.90))
print('95%: ', df_fl['roi_annual_avg_3y'].quantile(0.95))
print('99%: ', df_fl['roi_annual_avg_3y'].quantile(0.99))
print('99.9%: ', df_fl['roi_annual_avg_3y'].quantile(0.999))

In [ ]:
# Descriptive statistics for FL zipcodes 2018 sales price
print(df_fl['2018-04'].describe())
print('10%: ', df_fl['2018-04'].quantile(0.10))
print('20%: ', df_fl['2018-04'].quantile(0.20))
print('90%: ', df_fl['2018-04'].quantile(0.90))
print('95%: ', df_fl['2018-04'].quantile(0.95))

In [ ]:
# Plot boxplot for FL zipcodes annual ROI
# Create figure and set size
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,5)) 

# Plot price boxplot
sns.boxplot(df_fl['2018-04'], orient='v', color='steelblue', ax=ax[0]).set(
    xlabel='Median Sales Price') 

# Plot ROI boxplot
sns.boxplot(df_fl['roi_annual_avg_3y'], orient='v', color='green', ax=ax[1]
           ).set(xlabel='Median ROI')

# Set y-axis ranges and labels for subplots
ax[0].set(ylim=(0, 1000000))
ax[0].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
ax[0].set_ylabel('Dollars, $')
ax[1].set(ylim=(0,25))
ax[1].set_ylabel('Percent, %')

# Remove xticklabels and xticks
ax[0].set_xticklabels([])
ax[0].tick_params(bottom=False) 
ax[1].set_xticklabels([])
ax[1].tick_params(bottom=False) 

# Set title
plt.suptitle('Florida: Distribution of Sales Price and Annual ROI', y=1.02)
plt.tight_layout()

In [ ]:
# Scatterplot of FL sales price and annual ROI tailored for business case
# Create figure and set size
plt.figure(figsize=(12,6))

# Scatterplot of median price and roi
sns.scatterplot(data=df_fl, x='2018-04', y='roi_annual_avg_5y', color='lightblue')

# Plot 50th percentile sales price
plt.vlines(226100, 0, 40, color='black', linestyles='dashed',
          label='Median sales price ($)')

# Plot 50th percentile average annual 5yr ROI
plt.hlines(9.85, 0, 1000000, color='green', linestyles='dashed',
          label='Median annual ROI (%)')

# Set x-axis scale and ticks
plt.xlim(0,350000)
plt.xticks([50000,100000,150000,200000,250000,300000,350000,400000,450000,
            500000,550000,600000,650000,700000,750000,800000,850000,900000,
            950000,1000000],
           ['50k','100k','150k','200k','250k','300k', '350k', '400k', '450k',
           '500k', '550k','600k', '650k', '700k', '750k', '800k', '850k',
            '900k', '950k','1,000k'])
plt.xlabel('Median Sales Price ($)')

# Set y-axis scale and ticks
plt.ylim(0, 25)
plt.yticks([0, 5, 10, 15, 20, 25])
plt.ylabel('ROI, Annual (%)')

# Plot box highlighting target group with prices b/w 20th and 50th percentiles
# and ROI > 90th percentile and < 99th percentile
plt.vlines(160460, 15.995, 19.4, color='red', linestyles='dotted')
plt.hlines(15.995, 160460, 226100, color='red', linestyles='dotted')
#plt.hlines(17.87, 160460, 226100, color='red', linestyles='dotted')
plt.hlines(19.4, 160460, 226100, color='red', linestyles='dotted')

# Add and format legend
plt.legend(loc='upper right', frameon=False)

# Add title
plt.title('Florida: Median Sales Price vs Annual ROI')

# Remove borders
sns.despine()

plt.show()

## Data Preprocessing

In [ ]:
# Create subset df according to selection criteria:
# State of Florida 
# Sales price between 20th and 50th percentiles and 
# ROI > 90 percentile and < 99th percentiles (to remove 'hot market' outliers)
# Select top 10 zipcodes based on annual avg ROI over 5y period.
zillow_select = df_fl[(df_fl['2018-04'] >= 
                       np.percentile(df_fl['2018-04'], 20)) & 
                      (df_fl['2018-04'] < np.percentile(df['2018-04'], 50)) & 
                      (df_fl['roi_annual_avg_3y'] > 17.3) & 
                      (df['roi_annual_avg_3y'] < 23.2)].sort_values(
    by='roi_annual_avg_3y', ascending=False).head(10)
zillow_select

In [ ]:
# Create subset df according to selection criteria:  **SORT BY 3Y ANNUAL ROI TO GET ZIPCODES WITH UPWARD MOMENTUM?**
# State of Florida 
# Sales price between 20th and 50th percentiles and 
# ROI > 90 percentile and < 99th percentiles (to remove 'hot market' outliers)
# Select top 10 zipcodes based on annual avg ROI over 5y period.
zillow_select = df_fl[(df_fl['2018-04'] >= 
                       np.percentile(df_fl['2018-04'], 20)) & 
                      (df_fl['2018-04'] < np.percentile(df['2018-04'], 50)) & 
                      (df_fl['roi_annual_avg_5y'] > 15.955) & 
                      (df['roi_annual_avg_5y'] < 19.4)].sort_values(
    by='roi_annual_avg_5y', ascending=False).head(10)
zillow_select

### Convert to datetime

In [ ]:
# Convert date columns to datetime dtype
# Starter function
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[:], format='%Y-%m')

In [ ]:
# Select date columns only and pass them through function
zillow_select_date_df = zillow_select.iloc[:,8:-8]
zillow_select_date_df.columns = list(get_datetimes(zillow_select_date_df))
# Check dtype, which should be datetime
zillow_select_date_df.columns

In [ ]:
# Concat dfs (7 informational columns, 9 roi columns, date columns)
zillow_select_df = pd.concat([zillow_select.iloc[:, :7], zillow_select.iloc[:, -8:], zillow_select_date_df], axis=1)
zillow_select_df

In [ ]:
# check all columns
list(zillow_select_df.columns)

In [ ]:
# Create df for each zipcode selected for this phase of analysis
pierson = zillow_select_df[zillow_select_df['Zipcode']=='32180']
egyptlakeleto = zillow_select_df[zillow_select_df['Zipcode']=='33614']
stpete = zillow_select_df[zillow_select_df['Zipcode']=='33713']
orlando = zillow_select_df[zillow_select_df['Zipcode']=='32839']
bradenton = zillow_select_df[zillow_select_df['Zipcode']=='34205']
pompanobeach = zillow_select_df[zillow_select_df['Zipcode']=='33069']
jacksonville = zillow_select_df[zillow_select_df['Zipcode']=='32204']
westpalmbeach = zillow_select_df[zillow_select_df['Zipcode']=='33407']
lockhart = zillow_select_df[zillow_select_df['Zipcode']=='32810']
portcharlotte = zillow_select_df[zillow_select_df['Zipcode']=='33948']

In [ ]:
fl_list = [pierson, egyptlakeleto, stpete, orlando, bradenton,
           pompanobeach, jacksonville, westpalmbeach, lockhart, portcharlotte]

In [ ]:
# Check for missing data
stpete.isna().sum().sum()

### Reshape dataset

In [ ]:
# Reshape from wide to long format
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionID','Zipcode','City','State','Metro',
                                  'County','SizeRank','roi_cum','roi_13-14',
                                  'roi_14-15','roi_15-16','roi_16-17',
                                  'roi_17-18','roi_annual_avg_5y',
                                  'roi_annual_avg_3y'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
# Apply function to melt dfs
pierson_ts_full = melt_data(pierson)
egyptlakeleto_ts_full = melt_data(egyptlakeleto)
stpete_ts_full = melt_data(stpete)
orlando_ts_full = melt_data(orlando)
bradenton_ts_full = melt_data(bradenton)
pompanobeach_ts_full = melt_data(pompanobeach)
jacksonville_ts_full = melt_data(jacksonville)
westpalmbeach_ts_full = melt_data(westpalmbeach)
lockhart_ts_full = melt_data(lockhart)
portcharlotte_ts_full = melt_data(portcharlotte)

In [ ]:
# Create df with all melted city dfs and plot all series in one graph
# List of time series dfs
ts_list = [pierson_ts_full, egyptlakeleto_ts_full, stpete_ts_full, orlando_ts_full, 
           bradenton_ts_full, pompanobeach_ts_full, jacksonville_ts_full, 
           westpalmbeach_ts_full, lockhart_ts_full, portcharlotte_ts_full]

# Concat dfs into one df
ts_full_df = pd.concat(ts_list, axis=1)

# Rename columns
names = ['Pierson', 'Egypt Lake Leto', 'St. Petersburg', 'Orlando', 
             'Bradenton', 'Pompano Beach', 'Jacksonville', 'West Palm Beach',
            'Lockhart', 'Port Charlotte']
ts_full_df.columns = names

# Plot series
fig, ax = plt.subplots(figsize=(10,5))
ts_df.plot(ax=ax)
# Set x and y axis labels
plt.xlabel('Year')
plt.ylabel('Median House Price ($)')
# Format y-ticks
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

# Set title
plt.title('Florida: Median House Prices Over Time')
# Format legend
ax.legend(loc='upper left', frameon=False)
# Remove borders
sns.despine()

plt.show()

- Times series are not stationary. There appear to be cyclical and upward trends.

In [ ]:
pierson_ts.plot(kind='kde')

In [ ]:
# Filter time series to focus on post housing crisis period 2008-10
# pierson_ts = pierson_ts_full['2011':'2018']
# pierson_ts.plot()

## SARIMAX Modeling

### Train-test-split

In [ ]:
# Define function to apply train_test_split to all selected dfs
def tt_split(df_list, names):
    tts_list = []
    
    for i, df in enumerate(df_list):
        train_data = df[:-12] #all data, except last 12 months
        test_data = df[-12:] #last 12 months of data
        tts_list.extend([train_data, test_data])
        print(names[i],':', df.shape, ' // Train: ', train_data.shape,
             'Test: ', test_data.shape)
    return tts_list    #return list of train, test dfs

In [ ]:
tt_split(ts_list, names)

In [ ]:
pierson_train, pierson_test, egyptlakeleto_train, egyptlakeleto_test, \
    stpete_train, stpete_test, orlando_train, orlando_test, \
    bradenton_train, bradenton_test, pompanobeach_train, pompanobeach_test,\
    jacksonville_train, jacksonville_test, westpalmbeach_train, westpalmbeach_test,\
    lockhart_train, lockhart_test, portcharlotte_train, portcharlotte_test \
    = tt_split(ts_list, names)

In [ ]:
# Create train list and test list
train_dfs = [pierson_train, egyptlakeleto_train, stpete_train, orlando_train,
             bradenton_train, pompanobeach_train, jacksonville_train,
             westpalmbeach_train, lockhart_train, portcharlotte_train]
test_dfs = [pierson_test, egyptlakeleto_test, stpete_test, orlando_test,
             bradenton_test, pompanobeach_test, jacksonville_test,
             westpalmbeach_test, lockhart_test, portcharlotte_test]

### Define functions

In [ ]:
# Define function to plot autocorrelation function (ACF) and 
# partial autocorrelation function (PACF)

def plot_acf_pacf(ts):
    # Create figure, set format and size
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

    plot_acf(ts, ax[0]) #plot ACF
    ax[0].xaxis.set_major_locator(plt.MultipleLocator(2)) #set xtick frequency
    ax[0].set_xlabel('Lag') #set xlabel
    ax[0].set_ylabel('Autocorrelation') #set ylabel

    plot_pacf(pierson_ts, ax[1]) #plot PACF
    ax[1].set_xlim(-1,21) #set x-axis range
    ax[1].xaxis.set_major_locator(plt.MultipleLocator(1)) #set xtick frequency
    ax[1].set_xlabel('Lag') #set xlabel
    ax[1].set_ylabel('Partial Autocorrelation') #set ylabel

    plt.tight_layout()

In [ ]:
# Define function to check for stationarity
def stationarity_check(ts):
    """Calculate 12m rolling statistics and plot against original time series, 
    and perform and return Dickey Fuller test"""
    
    print('Results of Dickey-Fuller Test: \n')
        
    # Calculate rolling mean and standard deviation
    rolling_mean = ts.rolling(window=12, center=False).mean()
    rolling_std = ts.rolling(window=12, center=False).std()
        
    # Perform the Dickey Fuller Test
    df_test = adfuller(ts['value'])
    
    # Plot rolling statistics
    fig = plt.figure(figsize=(8,5))
    plt.plot(ts, color='black', label='Original')
    plt.plot(rolling_mean.dropna(), color='blue', label='Rolling Mean, 12-month', 
             linestyle='dotted')
    plt.plot(rolling_std.dropna(), color='green', 
             label = 'Rolling Std, 12-month',
             linestyle='dotted')
    
    # Format y-ticks
    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
    
    # Add legend
    plt.legend(loc='best')
    # Add title
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    
    # Print Dickey-Fuller test results
    print('Results of Dickey-Fuller Test: \n')
    df_output = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', 
                                             'Number of Lags Used', 
                                             'Number of Observations Used'])
    for key,value in df_test[4].items():
        df_output['Critical Value (%s)'%key] = value
    print(df_output)
    
    return None

In [ ]:
# Define function to fit baseline SARIMAX model and print results
# p = 1, d = 1 , m = 1, s=12

def fit_baseline_sarimax_model(metrics_df, name, train, order=(1,1,1),
                               seasonal_order=(0,0,0,12)):
    
    # Instantiate model
    arima_model = sm.tsa.statespace.SARIMAX(train,
                                           order=order,
                                           seasonal_order=seasonal_order)
    # Fit model
    output = arima_model.fit()
    
    # obtain comp metrics
    comp_metrics = [name, order, seasonal_order]
    # append AIC score
    comp_metrics.append(round(output.aic, 2))
    # add selected output metrics to df
    metrics_columns = ['Name', 'Baseline_Order', 'Baseline_Seasonal_Order','Baseline_AIC_Score']
    metrics_df = pd.DataFrame(comp_metrics).T
    metrics_df.columns = metrics_columns

    return metrics_df

    # Print results
    #display(output.summary())
    #output.plot_diagnostics(figsize=(11,8))
    
    #return output

In [ ]:
# Define function to find best parameters for SARIMAX model using auto_arima
"""parameters: m=12 months, trend='ct' (constant and linear), time series is 
seasonal and is not stationary"""

def find_params(name, ts):  
    params_output = pm.auto_arima(ts, start_p=1, start_q=1, max_p=4, max_q=4,
                                m=12, seasonal=True, stationary=False,
                                stepwise=True, trend='ct',
                                information_criterion='aic',  
                                suppress_warnings=True, trace=False, 
                                error_action='ignore')
   
    return name, params_output.order, params_output.seasonal_order, round(
        params_output.aic(),1)

In [ ]:
# Define function to fit SARIMAX model and print results
# p = 1, d = 1, m = 1 per ACF, PCF plots and ADF stationarity check

def fit_sarimax_model(ts, order=(1,1,1), seasonal_order=(0,0,0,12)):
    # Instantiate model
    arima_model = sm.tsa.statespace.SARIMAX(ts,
                                           order=order,
                                           seasonal_order=seasonal_order,
                                           enforce_stationarity=True,
                                           enforce_invertibility=False)
    # Fit model
    output = arima_model.fit()
    
    # Print results
    display(output.summary())
    output.plot_diagnostics(figsize=(11,8))
    
    return output

In [ ]:
def get_predictions(train, test, model_output, steps=24):

    # Calculate predictions from 2017-05 and calculate 95% confidence interval
    prediction = model_output.get_prediction(start=pd.to_datetime('2017-05'),
                                             end=pd.to_datetime('2018-04'),
                                            dynamic=True)
    pred_ci = prediction.conf_int(alpha=0.05)
    
    # Calculate forecast and 95% confidence interval for steps in future
    future = model_output.get_forecast(steps=steps, dynamic=True)
    future_ci = future.conf_int(steps=steps)

    # Create figure
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

    # Subplot 0: observed v predicted values
    # Plot observed values
    all_data = pd.concat([train, test], axis=0)
    all_data['2011':].plot(label='Actual', color='blue', ax=ax[0])

    # Plot predicted values
    prediction.predicted_mean.plot(color='green', 
                               label='Predicted', linestyle='dotted', 
                               ax=ax[0])

    # Plot the confidence interval
    ax[0].fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='gray', alpha=.15)

    # Format y-axis range
    ax[0].set_ylim(0,300000)
    # Set axes labels
    ax[0].set_ylabel('Mean Sales Price ($)')
    ax[0].set_xlabel('')
    # Format ticks
    ax[0].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

    # Subplot 1: forecasted values
    # Plot observed values
    all_data['2011':].plot(label='Actual', color='blue', ax=ax[1])
    # Plot forecasted values
    future.predicted_mean.plot(label='Forecast', color='darkorange', 
                           linestyle='dotted', ax=ax[1])

    # Prediction for forecast period and confidence interval
    forecast = future.predicted_mean[-1]
    maximum = future_ci.iloc[-1,1]
    minimum = future_ci.iloc[-1,0]
    predictions = {}
    predictions['forecast'] = forecast
    predictions['maximum'] = maximum
    predictions['minimum'] = minimum

    ax[1].fill_between(future_ci.index,
                    future_ci.iloc[:, 0],
                    future_ci.iloc[:, 1], color='gray', alpha=.15)
    
    # Format y-axis range
    ax[1].set_ylim(0,300000)
    # Set axes labels
    ax[1].set_ylabel('Mean Sales Price ($)')
    ax[1].set_xlabel('')
    # Format ticks
    ax[1].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

    # Set titles and legend
    plt.suptitle('Actual vs Predicted Mean Sales Price', y=1.02)
    ax[0].legend(['Actual', 'Predicted'], loc='upper left')
    ax[1].legend(['Actual', 'Predicted'], loc='upper left')
    
    plt.tight_layout()
    plt.show();
    
    # Calculate RMSE for last year of actual data vs prediction
    # Use test set for last year of data
    rmse = (np.sqrt((test.value - prediction.predicted_mean) ** 2)).mean()
    print('Root Mean Squared Error of predictions is ${}'.format(round(rmse, 2)))
    
    # Print forecasts
    #print('Forecasted values: ')
    #print(future.predicted_mean)

### ACF and PACF

In [ ]:
# Plot ACF and PCF for all time series
for i, train_df in enumerate(train_dfs): 
    plot_acf_pacf(train_df)

- Time series have comparable ACF and PACF plots.
- In ACF plot, previous sales price influences current sales price, but the significance decreases slowly.  
The first 18 lags are significant at 5% significance level (95% confidence interval).  
- The PACF plot indicates that partial autocorrelations for lags 0, 1, and 2 are statistically significant.  
Thus, the PACF suggests fitting a first- or second-order autoregressive (AR) model. The lag of 1  
has significant spike and then drops off.
- Autoregressive model is appropriate. According to PACF, the 'p' component could have a value of 1,  
which is value after which there is a sharp decrease.

### Stationarity

In [ ]:
# Check for stationarity
stationarity_check(pierson_train)

- p-value > 0.05 critical value. Time series is not stationary; there are trends in the data.

In [ ]:
# Difference the time series and recheck stationarity of differenced series
pierson_train_diff = pierson_train.diff(7).dropna()
stationarity_check(pierson_train_diff)

- With differencing factor of 7, p-value is < 0.05 critical value. Time series is stationary.

### Baseline model

In [ ]:
# Run baseline SARIMAX model for all dfs, with ar(1), d(1), and ma(1)
comp_results = []
for i, train_df in enumerate(train_dfs): 
    baseline_comp_df = fit_baseline_sarimax_model(metrics_df, names[i], train_df)   
    comp_results.append(baseline_comp_df)
    baseline_comp_df = pd.concat(comp_results)
print('Baseline Model')
baseline_comp_df

### Best parameters: optimal p,d,q

In [ ]:
# Run function to find best p,d,q,s parameters
comp_results_2 = []
    
for i, train_df in enumerate(train_dfs):
    best_comp_df = find_params(names[i], train_df)
    comp_results_2.append(best_comp_df)
    metrics_columns = ['Name','Best_Order', 'Best_Seasonal_Order', 'Best_AIC_Score']
    best_comp_df = pd.DataFrame(comp_results_2)
    best_comp_df.columns = metrics_columns
    
best_comp_df 

# metrics_columns = ['Name', 'Order', 'Seasonal_Order','AIC Score']
#     metrics_df = pd.DataFrame(comp_metrics).T
#     metrics_df.columns = metrics_columns

# comp_results = []
# for i, train_df in enumerate(train_dfs): 
#     baseline_comp_df = fit_baseline_sarimax_model(metrics_df, names[i], train_df)   
#     comp_results.append(baseline_comp_df)
#     baseline_comp_df = pd.concat(comp_results)
# print('Baseline Model')
# baseline_comp_df

In [ ]:
# concat df with comparative results
comparative_df = pd.merge(baseline_comp_df, best_comp_df, on='Name')
comparative_df

In [ ]:
comparative_df['AIC_Comp'] = comparative_df['Best_AIC_Score'] - comparative_df['Baseline_AIC_Score']
comparative_df

###  Model iterations

In [ ]:
# Run first zipcode through function to fit SARIMAX model
# Parameters obtained from best estimates given ACF and PACF plots, ADF stationarity check
pierson_output = fit_sarimax_model(pierson_train, order=(1,1,1), 
                                   seasonal_order=(0,0,0,12))

# predictions from model with inputs based on ACF, PACF plots
pierson_predictions = get_predictions(pierson_train, pierson_test, 
                                      pierson_output, steps=24)

In [ ]:
# Run time series through function to fit SARIMAX model using best params 
pierson_output_2 = fit_sarimax_model(pierson_train, order=(4,0,0), 
                                   seasonal_order=(0,0,2,12))

# predictions from model with inputs based on best_params auto_arima function
pierson_predictions_2 = get_predictions(pierson_train, pierson_test, 
                                        pierson_output_2, steps=24)

- Model with inputs from best params function has better distribution of residuals and lower RMSE than  
model with inputs inferred from ACF/PACF, despite them both having similar AIC scores.

In [ ]:
print(egyptlakeleto_train.info())
print(pierson_train.info())

In [ ]:
## Egypt Lake Leto
# Run time series through function to fit SARIMAX model using best params 
egyptlakeleto_output = fit_sarimax_model(egyptlakeleto_train, order=(4,0,2), 
                                   seasonal_order=(1,0,2,12))

# predictions from model with inputs based on best_params auto_arima function
egyptlakeleto_predictions = get_predictions(egyptlakeleto_train, egyptlakeleto_test, 
                                        egyptlakeleto_output, steps=24)

In [ ]:
## St. Petersberg
# Fit model using best_params
stpete_output = fit_sarimax_model(stpete_train, order=(0,0,0), 
                                   seasonal_order=(0,0,2,12))
# Get predictions
get_predictions(stpete_train, stpete_test, stpete_output, steps=24)

In [ ]:
## Orlando
# Fit model using best_params
orlando_output = fit_sarimax_model(orlando_train, order=(4,0,2), 
                                   seasonal_order=(2,0,2,12))
# Get predictions
get_predictions(orlando_train, orlando_test, orlando_output, steps=24)

In [ ]:
## Bradenton
# Fit model using best_params
bradenton_output = fit_sarimax_model(bradenton_train, order=(2,0,3), 
                                   seasonal_order=(0,0,2,12))
# Get predictions
get_predictions(bradenton_train, bradenton_test, bradenton_output, steps=24)

In [ ]:
egyptlakeleto_output = fit_sarimax_model(egyptlakeleto_train, order=(4,0,2), 
                                   seasonal_order=(2,0,2,12))

In [ ]:
## Pompano Beach
# Fit model using best_params
pompanobeach_output = fit_sarimax_model(pompanobeach_train, order=(4,0,2), 
                                   seasonal_order=(0,0,2,12))
# Get predictions
get_predictions(pompanobeach_train, pompanobeach_test, pompanobeach_output, steps=24)

In [ ]:
## Jacksonville
# Fit model using best_params
jacksonville_output = fit_sarimax_model(jacksonville_train, order=(2,0,4), 
                                   seasonal_order=(0,0,2,12))
# Get predictions
get_predictions(jacksonville_train, jacksonville_test, jacksonville_output, steps=24)

## Interpreting Results

## Recommendations

## Questions!

- price vs ROI as X variable?
- evaluation metric: RMSE
- ACF/PACF vs auto_arima or other function to determine best params?
- what if ACF, PACF params differ from auto_arima best params?
- differencing
- loop for baseline model and passing results to df


- RMSE for predictions vs observed and/or forecasts vs. what value?

- when/how to do train-test-split?

- function to loop through all ts? store all values in df?